# Предсказание трат по категориям

Всем привет! Мы команда <strong>MegaQuant</strong>

В этом ноутбуке мы представляем решение по 

<strong>Заданию 1: ПРЕДСКАЗАНИЕ ТРАТ ПО КАТЕГОРИЯМ</strong> для Хакатона в рамках 
\
<strong>Молодежной программы FINOdays</strong> Форума инновационных финансовых технологий FINOPOLIS 2021, 
\
организатором которого является Банк России. 

<strong>Задание:</strong> необходимо создать прототип сервиса, который с помощью машинного обучения сможет <strong>предсказывать траты</strong> пользователей «Тинькофф Банка» <strong>по категориям</strong> на следующий месяц. Данный сервис позволит улучшить систему кэшбэка. Высокое качество прогноза трат позволит рекомендовать каждому клиенту наиболее подходящие для него категории кэшбэка, которые будут отображаться в приложении в первую очередь.

Ниже приведено <strong>интерактивное содержание</strong>, по которому Вы можете быстро посмотреть наш код.

В ноутбуке есть <strong>описание</strong> каждой функции с комментариями.

## Содержание

- [Загрузка библиотек](#ch1)
- [Сбор и анализ данных](#ch2)
    - [Загрузка данных](#ch21)
    - [Анализ тренировочной выборки](#ch22)
      - [df_train](#ch221)
          - [Первичный анализ](#ch2211)
          - [Приведение категориальных признаков к числовому виду](#ch2212)
          - [Заполнение пропусков](#ch2213)
      - [df_socdem](#ch222)
          - [Первичный анализ](#ch2221)
          - [Приведение категориальных признаков к числовому виду](#ch2222)
          - [Заполнение пропусков](#ch2223)
      - [df_balance](#ch223)
          - [Первичный анализ](#ch2231)
          - [Заполнение пропусков](#ch2233)
      - [df_stories_texts](#ch224)
          - [Первичный анализ](#ch2241)
          - [Заполнение пропусков](#ch2243)
      - [df_stories_logs](#ch225)
          - [Первичный анализ](#ch2251)
          - [Приведение категориальных признаков к числовому виду](#ch2252)
          - [Заполнение пропусков](#ch2253)
      - [df_products](#ch226)
          - [Первичный анализ](#ch2261)
          - [Заполнение пропусков](#ch2263)
- [Feature Engineering](#ch3)
    - [Признак №1](#ch31)
    - [Признак №2](#ch32)
    - [Признак №3](#ch33)
    - [Признак №4](#ch34)
    - [Признак №5](#ch35)
    - [Признак №6](#ch36)
    - [Признак №7 (ARIMA)](#ch37)
- [Создание единой таблицы](#ch4)
- [Предсказательная модель](#ch5)
- [Метрики задач машинного обучения, соответствующие цели задачи](#ch6)
- [Тестовая выборка](#ch7)
    - [Общая идея](#ch71)
    - [Обработка данных](#ch72)
    - [Подсчет признаков для предсказания на первый месяц](#ch73)
    - [Подсчет признаков для предсказания на второй месяц](#ch74)
- [Тестирование модели на тестовых данных](#ch8)
- [Выбор бизнес-метрик](#ch9)
    - [Бизнес-метрика №1](#ch91)
        - [Подсчет бизнес-метрики](#ch911)
    - [Бизнес-метрика №2](#ch92)
        - [Подсчет бизнес-метрики](#ch921)
    - [Бизнес-метрика №3](#ch93)
    - [Дополнительная бизнес-метрика](#ch94)
- [Возможные улучшения](#ch10)
- [Вывод](#ch11)

## Загрузка библиотек  <a class="anchor" id="ch1"></a>

In [2]:
!pip install matplotlib
!pip install seaborn
!pip install plotly
!pip install sklearn
!pip install graphviz  
!pip install pdpbox
!pip install catboost
!pip install scipy
!pip install statsmodels
!pip install pingouin

In [3]:
# загрузка библиотек
from IPython import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
import datetime
from sklearn import preprocessing
import statistics
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
#sns.set_theme(style="whitegrid")
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

import random
random.seed = 42

In [4]:
def removing_outliers(column,frame, count = 0,total_outliers = 0, alpha = 1.5,max_iter = 5):
        """
        Метод удаления выбросов с помощью межквартильного размаха

        Параметры:
        - column: Имя столбца, где нужно найти выбросы
        - frame: Фрейм, где находится столбец для анализа
        - count: Количество рекурсивных циклов проверки 
        - total_outliers: Общая сумма выбросов, найденных за все циклы проверки
        - alpha: Коэффициент при вычислении интервала
        - max_iter: Максимальное количество циклов функции

        Возвращет:
        - frame: Обработанный фрейм
        - leng: Количество выбросов на цикле
        - total_outliers: Общее количество выбросов
        """
        # Получим квантиль 25%
        q25=np.array(frame[column].describe(percentiles=[.25,.75]).loc['25%'])
        # Получим квантиль 75%
        q75=np.array(frame[column].describe(percentiles=[.25,.75]).loc['75%'])
        # Получим первую границу
        first_part=q25-alpha*(q75-q25)
        # Получим 2 границу
        second_part=q75+alpha*(q75-q25)
        # Получим все значения, удовлетворяющие промежутку
        prep_frame = frame[(frame[column] > first_part) & (frame[column] < second_part)]
        # Подсчет количества удаленных выбросов
        leng = frame.shape[0] - prep_frame.shape[0]
        # Увеление счетчика циклов
        count += 1
        # Заменим исходную таблицу обработанной
        frame = prep_frame.copy()
        # Удалим ненужную таблицу
        del prep_frame
        # Проверим условия выхода
        if leng > 0 and count < max_iter:
            frame,leng,total_outliers = removing_outliers(column,frame, count,total_outliers+leng)
        else:
            print('Количество строк, выбранных для удаления {}: {}. Количество итераций {}'.format(column,total_outliers,count),end = "\n\n")
        return frame,leng,total_outliers

## Сбор и анализ данных  <a class="anchor" id="ch2"></a>

### Загрузка данных  <a class="anchor" id="ch21"></a>

In [ ]:
df_balance = pd.read_csv(r'C:\ВШЭ 3 курс\FINOdays\tinkoff_hackathon_data\tinkoff_hackathon_data\avk_hackathon_data_account_x_balance.csv')
df_products = pd.read_csv(r'C:\ВШЭ 3 курс\FINOdays\tinkoff_hackathon_data\tinkoff_hackathon_data\avk_hackathon_data_party_products.csv')
df_socdem = pd.read_csv(r'C:\ВШЭ 3 курс\FINOdays\tinkoff_hackathon_data\tinkoff_hackathon_data\avk_hackathon_data_party_x_socdem.csv')
df_stories_logs = pd.read_csv(r'C:\ВШЭ 3 курс\FINOdays\tinkoff_hackathon_data\tinkoff_hackathon_data\avk_hackathon_data_story_logs.csv')
df_story_texts = pd.read_csv(r'C:\ВШЭ 3 курс\FINOdays\tinkoff_hackathon_data\tinkoff_hackathon_data\avk_hackathon_data_story_texts.csv')
df_test = pd.read_csv(r'C:\ВШЭ 3 курс\FINOdays\tinkoff_hackathon_data\tinkoff_hackathon_data\avk_hackathon_data_test_transactions.csv')
df_train = pd.read_csv(r'C:\ВШЭ 3 курс\FINOdays\tinkoff_hackathon_data\tinkoff_hackathon_data\avk_hackathon_data_train_transactions.csv')

### Анализ тренировочной выборки  <a class="anchor" id="ch22"></a>

### df_train <a class="anchor" id="ch221"></a>

В данной таблице содержится информация о совершенных пользователями транзакциях, где
<ul>
<li><strong>party_rk</strong> - уникальный идентификационный номер клиента</li> 
<li><strong>account_rk</strong> - уникальный идентификационный номер счета</li>
<li><strong>financial_account_type_cd</strong> - тип карты (дебетовая/кредитовая)</li>
<li><strong>transaction_dttm</strong> - день совершения операции в формате ГГГГ-ММ-ДД</li>
<li><strong>transaction_amt_rur</strong> - сумма транзакции в рублях</li>
<li><strong>merchant_rk	</strong> - идентификационный номер мерчант-аккаунта (оборудования для приема платежей)</li>  
<li><strong>merchant_type</strong> - идентификационный номер типа мерчанта</li> 
<li><strong>merchant_group_rk</strong> - идентификационный номер мерчант-группы, обслуживающей соответствующий мерчант-аккаунт</li>
<li><strong>category	</strong> - категория транзакции</li>  
</ul>

In [ ]:
df_train.head()

Удалим колонки, связанные со счетом пользователя (account_rk). При входе в приложение "Тинькофф" в раздел, где можно выбрать категории для кэшбэка, пользователю предлагается выбрать 3 категории. Его выбор не зависит от его счета (account_rk), а зависит от того, с какого аккаунта он зашел (party_rk). Мы имеем право удалить колонки, связанные со счетом. 

In [ ]:
df_train = df_train.drop(['account_rk','merchant_rk','merchant_group_rk', 'merchant_type', 'transaction_type_desc'], axis = 1)

Переобозначим наименования категорий в числа 

In [ ]:
train_newnames = {cat_name: i for i,cat_name in enumerate(df_train['category'].unique())}  
df_train['category'].replace(train_newnames, inplace=True)   
# df_train_out['category'].replace(train_newnames, inplace=True)

##### Первичный анализ <a class="anchor" id="ch2211"></a>

In [ ]:
print('Количество строк: {}, количество столбцов: {}'.format(df_train.shape[0], df_train.shape[1]))

Посмотрим на типы переменных, чтобы корректно выполнять операции с ними. 

In [ ]:
df_train.info()

Переведем столбец с датами транзакций в тип datetime. 

In [ ]:
df_train['transaction_dttm'] = pd.to_datetime(df_train['transaction_dttm'])

Проверим, какие значения может принимать категориальный признак 'category'.

In [ ]:
df_train['category'].value_counts().reset_index()

Все значения категорий различные, нет недочетов в плане написания/лишних пробелов и тд.

Для более детального понимания данных, посмотрим на их основные характеристики. 

In [ ]:
df_train.describe().round(3)

In [ ]:
a = min(df_train['transaction_amt_rur'])
b = max(df_train['transaction_amt_rur'])
c = df_train['transaction_amt_rur'].mean()
print('Минимальное значение: {}, максимальное значение: {}, среднее значение: {}'.format(a, b, c))

Минимальное и максимальное значение транзакций сильно отличаются от среднего. Можно, предположить, что в столбце 'transaction_amt_rur' присутствуют выбросы.

##### Удаление выбросов <a class="anchor" id="ch2213"></a>

Есть несколько способов удаления выбросов: через z-score, через квантили и KMean. Но самый простой для понимания способ - через квантили (IQR), поэтому будем использовать его. 

In [ ]:
sns.boxplot(x=df_train['transaction_amt_rur']);

In [ ]:
df_train_out,_,_ = removing_outliers('transaction_amt_rur',df_train)

In [ ]:
print('Количество строк до удаления выбросов: {}, количество столбцов: {}'.format(df_train.shape[0], df_train.shape[1]))
print('Количество строк после удаления выбросов: {}, количество столбцов: {}'.format(df_train_out.shape[0], df_train_out.shape[1]))

In [ ]:
sns.boxplot(x=df_train_out['transaction_amt_rur']);

##### Заполнение пропусков <a class="anchor" id="ch2213"></a>

In [ ]:
print("\nКоличество пропущенных значений до заполнения пропусков:")
df_train_out.isna().sum().to_frame().sort_values(0, ascending = False).T

Для заполнения пропусков числовых переменных используются несколько методов, но мы будем использовать самый простой: в случае числового признака заменяем на среднее значение по столбцу (так как мы ранее удалили выбросы, то это среднее будет не смещенное), если это категориальный признак, то заменяем на моду. 

In [ ]:
df_train_out['transaction_amt_rur'] = df_train_out['transaction_amt_rur'].fillna(df_train_out['transaction_amt_rur'].mean())

In [ ]:
df_train_out['category'] = df_train_out['category'].fillna(df_train_out['category'].mode()[0])

In [ ]:
print("\nКоличество пропущенных значений после заполнения пропусков:")
df_train_out.isna().sum().to_frame().sort_values(0, ascending = False).T

##### Приведение категориальных признаков к числовому виду <a class="anchor" id="ch2212"></a>

Модели, которые мы будем в дальнейшем применять, работают только с числовыми признаками, поэтому приведем категориальные признаки к числовым. В основном различают 2 способа: One-hot Encoding и LabelEncoder. Второй применятся в том случае, когда мы можем выявить в данных определенный порядок, в нашем случае в столбце category ранжировать мы не можем, поэтому будем использовать One-hot Encoding.

In [ ]:
# функция для перевода категориального признака в числовой через One-hot Encoding
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

In [ ]:
df_train_out_num = encode_and_bind(df_train_out, 'category')

In [ ]:
del df_train_out_num['category']

In [ ]:
df_train_out_num

##### Визуализация данных

Общий объем транзакций по категориям в рублях:

In [ ]:
df_cluster = pd.merge(df_train, df_socdem)

In [ ]:
df_category_trans = df_cluster.groupby("category")['transaction_amt_rur'].sum().reset_index().sort_values(by = 'transaction_amt_rur', ascending = False)

In [ ]:
plt.figure(figsize=(10,7))
sns.set_style("ticks")
sns.barplot(x="transaction_amt_rur", y="category", data=df_category_trans, orient = 'h');
plt.xlabel('Общий объем транзакций')
plt.ylabel('Категория')
plt.title('Общий объем транзакций по категориям в рублях');

Распределение суммы операций за весь период по категориям пользователей

In [ ]:
# создадим вспомогательный столбец = первый день месяца в котором произошла операция 
df = df_train_out[['transaction_dttm','party_rk','transaction_amt_rur','category']] 
df['transaction_dttm'] = pd.to_datetime(df['transaction_dttm'] ) 
df['month_dttm'] = df['transaction_dttm'] + pd.offsets.Day() - pd.offsets.MonthBegin() 
 
# сделаем группировку сум_транзакций по месяцам 
df = df.groupby(['party_rk','category','month_dttm']).sum() 
df = df.reset_index()

In [ ]:
# ГРАФИК: Распределение суммы операций за весь период по категориям пользователей 
df_forploting = df.groupby(['party_rk','category']).sum().reset_index()

In [ ]:
f, ax = plt.subplots(figsize=(24, 8)) 
ax.set(xlabel="Общая сумма операций по категории",  
       ylabel = "Количество операций",   
       title=' Распределение суммы операций за весь период по категориям пользователей') 
 
sns.despine(f) 

sns.histplot(df_forploting, x="transaction_amt_rur", hue="category", element="step", multiple="stack", log_scale=True,bins=80)

### df_socdem <a class="anchor" id="ch222"></a>

В данной таблице содержится персональная информация клиентов, где
<ul>
<li><strong>party_rk</strong> - уникальный идентификационный номер клиента</li> 
<li><strong>gender_cd</strong> - пол клиента (F - женщина / M - мужчина / не указан)</li>
<li><strong>age</strong> - возраст клиента</li>
<li><strong>marital_status_desc</strong> - семейное положение (один из 7 вариантов: Женат/замужем / Вдовец, вдова / Холост/не замужем / Гражданский брак / Разведен (а) / Не проживает с супругом (ой) / Не указано)</li>
<li><strong>children_cnt</strong> - количество детей</li>
<li><strong>region_flg</strong> - бинарная характеристика региона клиента</li>
</ul>
<strong>Первичным ключом</strong> таблицы является атрибут party_rk.

In [ ]:
df_socdem.head()

##### Первичный анализ <a class="anchor" id="ch2221"></a>

In [ ]:
print('Количество строк: {}, количество столбцов: {}'.format(df_socdem.shape[0], df_socdem.shape[1]))

Проверим на всякий случай, что у нас party_rk это первичный ключ. 

In [ ]:
print('Количество уникальных значений party_rk: {}'.format(len(df_socdem['party_rk'].unique())))

In [ ]:
df_socdem.info()

In [ ]:
df_socdem.describe()

Заметим, что максимальное значение возраста может достигать значения 100, так что, вероятно, это может быть выбросом. Проверим это в разделе "Удаление выбросов". 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(9,4))

female = df_socdem[df_socdem['gender_cd'] == 'F']
male = df_socdem[df_socdem['gender_cd'] == 'M']

sns.distplot(female['age'], bins=12 ,ax=ax1)
sns.distplot(male['age'], bins=12, ax=ax2)

ax1.set_title('Age distr among females');
ax2.set_title('Age distr among males');

По графику можно сделать вывод, что возраст мужчин в среднем больше, чем возраст женщин. 

Добавим категориальный признак AgeGroup, позволяющий разделить всех клиентов на 6 групп в соответствии с их возрастом:

In [ ]:
df_socdem.loc[(df_socdem.age < 21),  'AgeGroup'] = '0-21'
df_socdem.loc[(21 <= df_socdem.age),  'AgeGroup'] = '21-29'
df_socdem.loc[(30 <= df_socdem.age),  'AgeGroup'] = '30-40'
df_socdem.loc[(41 <= df_socdem.age),  'AgeGroup'] = '41-54'
df_socdem.loc[(54 <= df_socdem.age),  'AgeGroup'] = '55-65'
df_socdem.loc[(65 <= df_socdem.age),  'AgeGroup'] = '65+'

In [ ]:
df_socdem

In [ ]:
# удалим ненужный столбец
del df_socdem['age']

##### Заполнение пропусков <a class="anchor" id="ch2223"></a>

In [ ]:
print("\nКоличество пропущенных значений до заполнения пропусков:")
df_socdem.isna().sum().to_frame().sort_values(0, ascending = False).T

Вместо пропущенных значений в колонках 'marital_status_desc' и 'gender_cd' запишем значение unknown, что будет означать, что пользователь не указала своего статуса или пола. 

In [ ]:
df_socdem['gender_cd'] = df_socdem['gender_cd'].fillna('unknown')
df_socdem['marital_status_desc'] = df_socdem['marital_status_desc'].fillna('unknown')

In [ ]:
print("\nКоличество пропущенных значений после заполнения пропусков:")
df_socdem.isna().sum().to_frame().sort_values(0, ascending = False).T

##### Приведение категориальных признаков к числовому виду <a class="anchor" id="ch2222"></a>

Заменим признак AgeGroup на числовые значения с помощью LabelEncoder, так как внутри этого признака сожно выявить порядок. 

In [ ]:
# AgeGroup - категориальный признак, переведем его в тип int
le_age = preprocessing.LabelEncoder()
le_age.fit(df_socdem['AgeGroup'])
df_socdem['AgeGroup'] = le_age.transform(df_socdem['AgeGroup'])

Остальные колонки 'gender_cd' и 'material_status_desc' заменим при помощи One-hot Encoding. 

In [ ]:
df_socdem_num = encode_and_bind(df_socdem, 'gender_cd')
df_socdem_num_ = encode_and_bind(df_socdem_num, 'marital_status_desc')

In [ ]:
del df_socdem_num_['marital_status_desc']
del df_socdem_num_['gender_cd']

In [ ]:
df_socdem_num_.head()

Визуализируем данные:

Распределение клиентов по полу:

In [ ]:
sns.set(rc={'figure.figsize':(8,5)})
ax = sns.countplot(y="gender_cd", data=df_socdem_num, palette="deep")
ax.set_title("Распределение по полу")
ax.set(xlabel="Количество пользователей", ylabel = "Пол");

Распределение клиентов по возрасту:

In [ ]:
df_socdem_graph = df_socdem.copy()
df_socdem_graph.loc[(df_socdem_graph.age < 21),  'AgeGroup'] = '0-21'
df_socdem_graph.loc[(21 <= df_socdem_graph.age),  'AgeGroup'] = '21-29'
df_socdem_graph.loc[(30 <= df_socdem_graph.age),  'AgeGroup'] = '30-40'
df_socdem_graph.loc[(41 <=  df_socdem_graph.age),  'AgeGroup'] = '41-54'
df_socdem_graph.loc[(54 <= df_socdem_graph.age),  'AgeGroup'] = '55-65'
df_socdem_graph.loc[(65 <= df_socdem_graph.age),  'AgeGroup'] = '65+'

In [ ]:
sns.set(rc={'figure.figsize':(8,5)})
ax = sns.countplot(y="AgeGroup", data=df_socdem_graph, palette="deep")
ax.set_title("Распределение по возрасту")
ax.set(xlabel="Количество пользователей", ylabel = "Возрастные группы");

Распределение клиентов по статусу:

In [ ]:
sns.set(rc={'figure.figsize':(8,5)})
ax = sns.countplot(y="marital_status_desc", data=df_socdem_graph, palette="deep")
ax.set_title("Распределение по статусу")
ax.set(xlabel="Количество пользователей", ylabel = "Статусы");

Распределение по сумме транзакций:

In [ ]:
df_train_graph = df_train_out[['party_rk', 'transaction_amt_rur']]
bins = [0, 200, 400, 600, 800, 1000]
labels = ["0-200","200-400","400-600", '600-800', '800-1000']
df_train_graph['transaction_categories'] = pd.cut(df_train_graph['transaction_amt_rur'], bins = bins, labels = labels)

In [ ]:
sns.set(rc={'figure.figsize':(8,5)})
ax = sns.countplot(y="transaction_categories", data=df_train_graph, palette="deep")
ax.set_title("Распределение по сумме транзакций")
ax.set(xlabel="Количество пользователей", ylabel = "Сумма транзакций");

### df_balance <a class="anchor" id="ch223"></a>

В данной таблице содержится ежемесячная информация о изменениях баланса клиентов, где
<ul>
<li><strong>party_rk</strong> - уникальный идентификационный номер клиента</li> 
<li><strong>account_rk</strong> - уникальный идентификационный номер счета</li>
<li><strong>prev_month</strong> - дата последнего дня (t-1)-го месяца в формате ГГГГ-ММ-ДД</li>
<li><strong>cur_month</strong> - дата последнего дня t-го месяца в формате ГГГГ-ММ-ДД</li>
<li><strong>balance_chng</strong> - изменение баланса в течение периода времени [prev_month, cur_month]</li>
</ul>
<strong>Первичный ключ</strong> таблицы является составным из двух атрибутов: account_rk + prev_month.

In [ ]:
df_balance.head()

##### Первичный анализ <a class="anchor" id="ch2231"></a>

In [ ]:
print('Количество строк: {}, количество столбцов: {}'.format(df_balance.shape[0], df_balance.shape[1]))

In [ ]:
df_balance.info()

Переведем признаки 'prev_month' и 'cur_month' в тип datetime.

In [ ]:
df_balance['prev_month'] = pd.to_datetime(df_balance['prev_month'])
df_balance['cur_month'] = pd.to_datetime(df_balance['cur_month'])

In [ ]:
df_balance

##### Заполнение пропусков <a class="anchor" id="ch2233"></a>

In [ ]:
print("\nКоличество пропущенных значений до заполнения пропусков:")
df_balance.isna().sum().to_frame().sort_values(0, ascending = False).T

В тех строчках, где у нас пропущенные значения 'prev_month' и 'cur_month', нет смысла вставлять туда  моду, так как мы работаем с датами.

In [ ]:
df_balance = df_balance.dropna(subset=["prev_month"],axis =0)
df_balance = df_balance.dropna(subset=["cur_month"],axis = 0)

In [ ]:
print("\nКоличество пропущенных значений до заполнения пропусков:")
df_balance.isna().sum().to_frame().sort_values(0, ascending = False).T

##### Удаление выбросов <a class="anchor" id="ch2233"></a>

In [ ]:
sns.boxplot(x=df_balance['balance_chng']);

In [ ]:
df_balance['pos_change'] = df_balance['balance_chng'].apply(lambda x: x if x >= 0 else x * -1)
df_balance_out,_,_ = removing_outliers('pos_change',df_balance)
df_balance_out=df_balance_out.drop(['pos_change'],axis = 1)
df_balance=df_balance.drop(['pos_change'],axis = 1)

In [ ]:
# Q1 = df_balance['balance_chng'].quantile(0.25)
# Q3 = df_balance['balance_chng'].quantile(0.75)
# IQR = Q3 - Q1
# df_balance_out = df_balance[~((df_balance['balance_chng'] < (Q1 - 1.5 * IQR)) | (df_balance['balance_chng'] > (Q3 + 1.5 * IQR)))]

In [ ]:
print('Количество строк до удаления выбросов: {}, количество столбцов: {}'.format(df_balance.shape[0], df_balance.shape[1]))
print('Количество строк после удаления выбросов: {}, количество столбцов: {}'.format(df_balance_out.shape[0], df_balance_out.shape[1]))

In [ ]:
sns.boxplot(x=df_balance_out['balance_chng']);

In [ ]:
df_balance_out.head()

### df_stories_texts <a class="anchor" id="ch224"></a>

В данной таблице содержится информация об историях, выкладываемых Тинькоффом, где
<ul>
<li><strong>story_id</strong> - уникальный идентификационный номер истории</li> 
<li><strong>name</strong> - заголовок истории</li>
<li><strong>story_text</strong> - текст истории</li>
</ul>
<strong>Первичным ключом</strong> таблицы является атрибут story_id.

In [ ]:
df_story_texts.head()

##### Первичный анализ <a class="anchor" id="ch2241"></a>

In [ ]:
print('Количество строк: {}, количество столбцов: {}'.format(df_story_texts.shape[0], df_story_texts.shape[1]))

In [ ]:
df_story_texts.info()

##### Заполнение пропусков <a class="anchor" id="ch2243"></a>

In [ ]:
print("\nКоличество пропущенных значений:")
df_story_texts.isna().sum().to_frame().sort_values(0, ascending = False).T

In [ ]:
df_story_texts['name'] = df_story_texts['name'].fillna('Unknown')

In [ ]:
print("\nКоличество пропущенных значений:")
df_story_texts.isna().sum().to_frame().sort_values(0, ascending = False).T

In [ ]:
print('Всего топиков стори: {}'.format(df_story_texts['name'].nunique()))

### df_stories_logs <a class="anchor" id="ch225"></a>

В данной таблице содержится информация о действиях клиентов относительно историй, где
<ul>
<li><strong>party_rk</strong> - уникальный идентификационный номер клиента</li> 
<li><strong>date_time</strong> - дата и время действия клиента в формате ГГГГ-ММ-ДД ЧЧ:ММ:СС </li>
<li><strong>story_id</strong> - уникальный идентификационный номер истории</li>
<li><strong>category</strong> - уникальный идентификационный номер категории действия</li>
<li><strong>event</strong> - наименование действия (like / dislike / ...)</li>
</ul>
<strong>Первичный ключ</strong> таблицы является составным из двух атрибутов: party_rk + date_time.

In [ ]:
df_stories_logs.head()

##### Первичный анализ <a class="anchor" id="ch2251"></a>

In [ ]:
print('Количество строк: {}, количество столбцов: {}'.format(df_stories_logs.shape[0], df_stories_logs.shape[1]))

In [ ]:
df_stories_logs.info()

Заметим, что у признака 'date_time', который представляет из себя дату, тип object, хотя должен быть тип datetime.

In [ ]:
df_stories_logs['date_time'] = pd.to_datetime(df_stories_logs['date_time'])

In [ ]:
df_stories_logs['event'].value_counts().reset_index()

Удалим строки с непонятными (числовыми) значениями по признаку 'index' 

In [ ]:
df_stories_logs = df_stories_logs[(df_stories_logs['event'] == 'like') | (df_stories_logs['event'] =='dislike') | (df_stories_logs['event'] =='favorite')] 

In [ ]:
df_stories_logs.head()

##### Приведение категориальных признаков к числовому виду <a class="anchor" id="ch2252"></a>

In [ ]:
# заменим в стобце event категориальные значение на числа
df_stories_logs['event_cat'] = np.where(df_stories_logs['event'] == 'dislike' , 0, df_stories_logs['event'])
df_stories_logs['event_cat'] = np.where(df_stories_logs['event'] == 'like' , 1, df_stories_logs['event_cat'])
df_stories_logs['event_cat'] = np.where(df_stories_logs['event'] == 'favorite' , 2, df_stories_logs['event_cat'])

In [ ]:
df_stories_logs.head()

In [ ]:
del df_stories_logs['event']

##### Заполнение пропусков <a class="anchor" id="ch2253"></a>

In [ ]:
print("\nКоличество пропущенных значений:")
df_stories_logs.isna().sum().to_frame().sort_values(0, ascending = False).T

### df_products <a class="anchor" id="ch226"></a>

В данной таблице содержится информация о связи пользователей с некоторыми категориями продуктов, где
<ul>
<li><strong>party_rk</strong> - уникальный идентификационный номер клиента</li> 
<li><strong>product1 - ... - product7</strong> - связи пользователя и соответствующего продукта </li>
</ul>
<strong>Первичным ключом</strong> таблицы является атрибут party_rk.

In [ ]:
df_products.head()

##### Первичный анализ <a class="anchor" id="ch2261"></a>

In [ ]:
print('Количество строк: {}, количество столбцов: {}'.format(df_products.shape[0], df_products.shape[1]))

In [ ]:
df_products.info()

Проверим, являются ли значения во всех столбцах бинарными. 

In [ ]:
columns_products = df_products.columns[1:]
for column in columns_products:
    print('Столбец: ',column)
    print(df_products[column].value_counts(),end='\n\n')

Да, все признаки являются бинарными. 

##### Заполнение пропусков <a class="anchor" id="ch2263"></a>

In [ ]:
print("\nКоличество пропущенных значений:")
df_products.isna().sum().to_frame().sort_values(0, ascending = False).T

## Feature Engineering  <a class="anchor" id="ch3"></a>

Создадим <strong> новые признаки </strong> для единой итоговой таблицы, характеризующие совершаемые клиентами транзакции в различных категориях. Новые характеристики будут отражать  
<ul>
    <li>средние траты по категориям,</li>
    <li>частоту покупок (в т.ч. повторных покупок в одной категории), </li>
    <li>статистику активности клиентов в интернете по данным категориям товаров и т.д.</li>
</ul>

### Признак №1 <a class="anchor" id="ch31"></a>

$\frac{\mbox{(Количество лайков у человека в конкретной категории в данном месяце)}}{\mbox{(Общее количество лайков человека в конкретном месяце)}}$

<strong>Признак:</strong> рейтинг категорий, которые лайкнул пользователь. Для того, чтобы его посчитать, смотрим на то, сколько лайков юзер поставил на стори той или иной категории и делим это на суммарное количество лайков по всем категориям. 

In [ ]:
def create_df_user_cat_action(matrix3, matrix4, action):
    '''
    FUNC: считает количество действий (лайков, дизлайков, добавление в любимые) в конкретной категории для каждого пользователя
    IN: matrix3 (информация о пользователях)
    IN: matrix4 (информация о действиях пользователей)
    OUT: df_cat_likes_1 таблица, содержащая информацию о количестве лайков пользователя в каждой категории
    '''
    # создадим список party_ids, длиной в кооличество категорий * количество уникальных пользователей
    party_ids = []
    for i in matrix3['party_rk'].unique():
        for j in range(81):
            party_ids.append(i)
    cat_action = list(matrix4['category'].unique()) * 50000
    temple_action = pd.DataFrame({'party_rk': party_ids, 'category': cat_action})
    
    # выберем данные, соответствующие последним двум месяцам и определенному действию пользователей
    matrix4['Month'] = pd.DatetimeIndex(matrix4['date_time']).month
    df_stories_logs_action = matrix4[((matrix4['Month'] == 10) | (matrix4['Month'] == 11)) &
                                       (matrix4['event_cat'] == action)]
    
    # оставим только нужные нам столбцы
    df_stories_logs_new_action = df_stories_logs_action.copy()
    del df_stories_logs_new_action['date_time']
    del df_stories_logs_new_action['story_id']
    del df_stories_logs_new_action['Month']
    
    # создадим столбец 'number_action', в котором будут храниться количество дейстий, совершенных пользователем по конкректной категории
    df_cat_action = pd.DataFrame(df_stories_logs_new_action.groupby(["party_rk", "category"])['event_cat'].count())
    df_cat_action.columns = ['number_action']
    df_cat_action.reset_index(inplace = True)
    df_cat_action_1 = pd.merge(temple_action, df_cat_action, how='left')
    df_cat_action_1['number_action'] = df_cat_action_1['number_action'].fillna(0)
    
    return df_cat_action_1


def create_df_user_cat_sum_action(temp1):
    '''
    FUNC: считает суммарное количество действий каждого пользователя
    IN: temp1 (информация о количестве действий пользователей в конкректных категориях)
    OUT: df_cat_likes_likes_all_cat (информация о суммарном количестве действий каждого пользователя)
    '''
    
    # группируем таблицу по 'party_rk' и суммируем количество действий для каждого пользователя
    df_cat_likes_likes_all_cat = temp1[['party_rk', 'number_action']]
    df_cat_likes_likes_all_cat.columns = ['party_rk', 'sum_number_action']
    df_cat_likes_likes_all_cat = df_cat_likes_likes_all_cat.groupby(['party_rk'])['sum_number_action'].agg('sum').reset_index()
    
    return df_cat_likes_likes_all_cat


def create_df_freq_user_action(temp2, temp3):
    '''
    FUNC: считает частоту определенного действия каждого пользователя в конкретной категории
    IN: temp2 (таблица с количеством действий каждого пользователя в каждой категории)
    IN: temp3 (таблица с суммарным количеством действий каждого пользователя)
    OUT: таблица с частотой действий пользователя в каждой категории
    '''
    # соединяем две таблицы по 'party_rk' и добавляем новый столбец с частотой определнного действия для каждого пользователя в каждой категории
    df_temp1_likes = pd.merge(temp2, temp3, how = 'left')
    df_temp1_likes['freq_action'] = 1.0 * df_temp1_likes['number_action']/df_temp1_likes['sum_number_action']
    df_temp1_likes = df_temp1_likes[['party_rk', 'category', 'freq_action']]
    df_temp1_likes.fillna(0, inplace = True)
    
    return df_temp1_likes

In [ ]:
def create_final_df(matrix3, matrix4, temp_fin):
    '''
    FUNC: создает таблицу, где строки будут пользователями, а столбцы - категориями, на их переесечении - количество действий 
    каждого пользователя в конкректной категории
    IN: matrix3 (информация о пользователях, включая идентификатор пользователя)
    IN: matrix4 (информация о дейтсвиях пользователя)
    IN: temp_fin (информация о частоте действий пользователей в конкректной категории)
    OUT:
    '''
    
    df_new_action = pd.DataFrame({'party_rk': matrix3['party_rk'].unique()})
    for i in matrix4['category'].unique():
        df_temp_for_action = temp_fin[temp_fin['category'] == i]
        del df_temp_for_action['category']
        df_new_action_ = pd.merge(df_new_action, df_temp_for_action, left_on = 'party_rk', right_on = 'party_rk', how = 'left')
        df_new_action = df_new_action_.copy()
    
    return df_new_action

In [ ]:
df_user_cat_likes = create_df_user_cat_action(df_socdem_num_, df_stories_logs, 1)
df_user_cat_likes.head()

In [ ]:
df_user_cat_sum_likes = create_df_user_cat_sum_action(df_user_cat_likes)
df_user_cat_sum_likes.head()

In [ ]:
df_freq_user_likes  = create_df_freq_user_action(df_user_cat_likes, df_user_cat_sum_likes)
df_freq_user_likes.head()

In [ ]:
final_df_likes = create_final_df(df_socdem_num_, df_stories_logs, df_freq_user_likes)
final_df_likes.head()

In [ ]:
s_likes = ['party_rk']
for i in df_stories_logs['category'].unique():
    string_name_likes  = 'storyLikes_' + str(i) 
    s_likes.append(string_name_likes)
final_df_likes.columns = s_likes
df_new_likes_final = final_df_likes.copy()

In [ ]:
df_new_likes_final.head()

In [ ]:
f1 = df_new_likes_final
f1

#### Визуализируем признак

Заметим, что <strong>55 категорий</strong> историй из таблицы df_stories_logs ('category') связаны по смыслу с <strong>37-ью категориями</strong> товаров из таблиц df_train и df_test о транзакциях ('category'). Присвоим каждой из 55-ти категорий одну из категорий среди 37-ми и запишем результат в файл <strong>df_category81_category36.csv</strong>. Загрузим этот файл и будем пользоваться им далее:

In [ ]:
df_cat_stor  = pd.read_csv(r'C:\ВШЭ 3 курс\FINOdays\tinkoff_hackathon_data\tinkoff_hackathon_data\df_category81_category36.csv')

In [ ]:
df_graph_action = pd.merge(df_stories_logs, df_cat_stor, left_on = 'category', right_on = 'category81')
df_graph_action = df_graph_action[['party_rk', 'event_cat', 'category36']]
df_graph_action['event_'] = np.where(df_graph_action['event_cat'] == 0 , 'dislike', df_graph_action['event_cat'])
df_graph_action['event_'] = np.where(df_graph_action['event_cat'] == 1 , 'like' , df_graph_action['event_'])
df_graph_action['event_'] = np.where(df_graph_action['event_cat'] == 2 , 'favorite', df_graph_action['event_'])
df_graph_action['1'] = [1] * len(df_stories_logs)
df_fin = pd.DataFrame(df_graph_action.groupby(['party_rk', 'event_', 'category36'])['1'].agg('sum')).reset_index()
table = pd.pivot_table(df_fin, values='1', index=['category36'],
                    columns=['event_'], aggfunc=np.sum, fill_value=0).reset_index()

In [ ]:
plt.rcParams["figure.figsize"] = (10,5)
table.plot(x='category36', kind='bar', stacked=True,
        title='Действия пользователя по категориям');
plt.ylabel('Количество действий')
plt.xlabel('Категория');

### Признак №2 <a class="anchor" id="ch32"></a>

<strong>Признак:</strong> рейтинг категорий, которые дизлайкнул пользователь. Для того, чтобы его посчитать, смотрим на то, сколько дизлайков юзер поставил на стори той или иной категории и делим это на суммарное количество дизлайков по всем категориям. 

$\frac{\mbox{(Количество дизлайков у человека в конкретной категории в данном месяце)}}{\mbox{(Общее количество дизлайков человека в конкретном месяце)}}$

In [ ]:
df_user_cat_dislikes = create_df_user_cat_action(df_socdem_num_, df_stories_logs, 0)
df_user_cat_dislikes.head()

In [ ]:
df_user_cat_sum_dislikes = create_df_user_cat_sum_action(df_user_cat_dislikes)
df_user_cat_sum_dislikes.head()

In [ ]:
df_freq_user_dislikes  = create_df_freq_user_action(df_user_cat_dislikes, df_user_cat_sum_dislikes)
df_freq_user_dislikes.head()

In [ ]:
final_df_dislikes = create_final_df(df_socdem_num_, df_stories_logs, df_freq_user_dislikes)
final_df_dislikes.head()

In [ ]:
s_dislikes = ['party_rk']
for i in df_stories_logs['category'].unique():
    string_name_dislikes = 'storydislikes_' + str(i) 
    s_dislikes.append(string_name_dislikes)
final_df_dislikes.columns = s_dislikes
df_new_dislikes_final = final_df_dislikes.copy()

In [ ]:
df_new_dislikes_final.head()

In [ ]:
f2 = df_new_dislikes_final
f2

### Признак №3 <a class="anchor" id="ch33"></a>

<strong>Признак:</strong> средняя сумма в месяц, потраченная пользователем на конкретную категорию (подсчет по последним трем месяцам).

Получим 37 новых столбцов для единой итоговой таблицы.

Укажем даты начала и конца периода из крайних трех месяцев тренировочной выборки:

In [ ]:
def cutDate(start_date, stop_date, matrix):
    '''
    FUNC: выводит транзакции в указанном периоде
    IN: start_date (дата, с которой начинается необходимый временной промежуток (не включительно))
    IN: stop_date (дата, на которой заканчивается необходимый временной промежуток (включительно))
    IN: matrix (таблица с информацией о транзакциях)
    OUT: pre_cut (таблица транзакций за указанный период)
    OUT: final_cut (таблица транзакций за указанный период, ключающая только столбец 'party_rk')
    '''
        
    temp1 = matrix[matrix['transaction_dttm'] <= stop_date]
    pre_cut = temp1[start_date < temp1['transaction_dttm']]
    final_cut = pre_cut['party_rk'].to_frame()
    return pre_cut, final_cut
    
    
def feature3(matrix, matrix3, date1, date4):
    '''
    FUNC: создает таблицу со значениями искомого признака
    IN: matrix (таблица с информацией о транзакциях)
    IN: matrix3 (таблица с информацией о пользователях)
    IN: date1 (дата окончания  рассматриваемого периода времени (включительно))
    IN: date4 (дата начала рассматриваемого периода времени (не включительно))
    OUT: final_feature3 (таблица, строки которой - пользователи, столбцы - категории, на пересечении - искомый признак)
    '''

    # Выпишем все party_rk всех пользователей
    df_pr3 = matrix3['party_rk'].to_frame()

    # Выведем все транзакции, совершаемые в периоде времени [date4, date1]
#     month_3_pre_cut, month_3_final_cut  = cutDate(date4, date1, matrix)
    
    temp1 = matrix[matrix['transaction_dttm'] <= date1]
    month_3_pre_cut = temp1[date4 <  temp1['transaction_dttm']]
    month_3_final_cut = month_3_pre_cut['party_rk'].to_frame()
    
    # Пройдем по каждой из 37 категорий и посчитаем среднюю сумму трат, сгруппированных по пользователям, 
    # а затем занесем новый получившийся столбец в общую таблицу df_pr3

    res3 = pd.DataFrame()
    for i in range(37):
        temp2 = month_3_pre_cut[month_3_pre_cut['category'] == i]
        temp3 = temp2[['party_rk', 'transaction_amt_rur']]
        feature3 = temp3.groupby(['party_rk']).mean()
        name2 = 'categoryii' + str(i)+'_mean3month'
        feature3 = feature3.rename(columns={'transaction_amt_rur': name2}) 
        df_pr3 = pd.merge(df_pr3, feature3, left_on='party_rk', right_on='party_rk', how='left')
    
    # Заполним нулями все поля для пользователей, которые ничего не покупали в рассматриваемой категории за данные 3 месяца
    final_feature3 = df_pr3.fillna(0)
    
    return final_feature3

In [ ]:
f3 = feature3(df_train_out, df_socdem_num, datetime.datetime(2019,11,30), datetime.datetime(2019,8,31))
f3

### Признак №4 <a class="anchor" id="ch34"></a>

<strong>Признак 1: </strong> количество месяцев с транзакциями по категориям

<strong>Признак 2: </strong> стандартное отклонение суммы траназкций в месяц по категориям (посчитано для месяцев с транзакциями)

<strong>Признак 3: </strong> средняя сумма транзакций в месяц по категориям (посчитано для месяцев с транзакциями)


Получим 37*3+1 новых столбцов

In [ ]:
def calc_trans_by11month(matrix):
    '''
    FUNC: считает суммы транзакций за каждый месяц по всем юзерам
    IN: matrix (таблица с информацией о транзакциях)
    OUT: cat_ptable ([{юзер,месяц} сумма_месяца_Категория1,...,сумма_месяца_Категория37])
    '''

    # создадим вспомогательный столбец = первый день месяца в котором произошла операция
    df = matrix[['transaction_dttm','party_rk','transaction_amt_rur','category']]
    df['transaction_dttm'] = pd.to_datetime(df['transaction_dttm'] )
    df['month_dttm'] = df['transaction_dttm'] + pd.offsets.Day() - pd.offsets.MonthBegin()

    # уберем все наблюдения последнего месяца
    df = df[df['month_dttm'] < max(df['month_dttm'])]

    # сделаем группировку сум_транзакций по месяцам
    df = df.groupby(['party_rk','category','month_dttm']).sum()
    df = df.reset_index()
    
    # создадим cat_ptable 
    cat_ptable = pd.pivot_table(df, values=['transaction_amt_rur'], columns=['category'],  index=['party_rk','month_dttm'], aggfunc={np.sum})
    cat_ptable.columns = cat_ptable.columns.get_level_values(2)
    cat_ptable = cat_ptable.reset_index()
    return cat_ptable

def createFeature4(cat_ptable, matrix3):
    '''
    FUNC: считает features 
    IN: cat_ptable ([{юзер,месяц} сумма_месяца_Категория1,...,сумма_месяца_Категория37])
    IN: matrix3 (таблица с информацией о пользователях)
    OUT: feature_all
    '''
    # заполним дефолтными np.NaN суммы_расходов_категорий пользователей, которые пропали после чистки или удаления 12го месяца 
    #   в итоге, должно получиться len(df_socdem) == 50 000 штук наблюдений 
    lost_users = list(set(matrix3.party_rk)-set(cat_ptable.party_rk))
    row = cat_ptable.iloc[:1] 
    for user_id in lost_users:
        row.party_rk = user_id
        row.iloc[:,2:row.shape[1]] = np.NaN
        cat_ptable = cat_ptable.append(row, ignore_index = True)
    
    grouped_ptabel = cat_ptable.groupby(['party_rk'])
    # посчитаем кол-во месяцев с операциями OUT:feature_count
    feature_count = grouped_ptabel.count()
    # поставим суфикс '_count'в columns
    feature_count.columns = pd.Index(list(['categoryii'+str(i)+'_count' for i in list(feature_count.columns)]))
    
    grouped_ptabel = cat_ptable.fillna(0).groupby(['party_rk'])
    # посчитаем среднее операций OUT:feature_mean
    feature_mean = grouped_ptabel.mean()
    # поставим суффикс '_mean'в columns
    feature_mean.columns = pd.Index(list(['categoryii'+str(i)+'_mean' for i in list(feature_mean.columns)]))

    # посчитаем среднее операций OUT:feature_std
    feature_std = grouped_ptabel.std()
    # поставим суффикс '_std'в columns
    feature_std.columns = pd.Index(list(['categoryii'+str(i)+'_std' for i in list(feature_std.columns)]))

    # объединим в одну таблицу 
    feature_all = (feature_count
                                .reset_index().merge(feature_mean.reset_index(), on='party_rk', how='left')
                                              .merge(feature_std.reset_index(), on='party_rk', how='left'))
    return feature_all


cat_ptable = calc_trans_by11month(df_train)
f4 = cat_features = createFeature4(cat_ptable,df_socdem_num)
f4

### Признак №5 <a class="anchor" id="ch35"></a>

<strong>Признак:</strong> Частота повторных покупок у одного человека по каждой из категорий. 

<strong>Повторными</strong> будем называть <strong>покупки</strong> одной категории, совершенные друг за другом. 

Для <strong>нахождения этого признака</strong> для j-го пользователя по i-ой категории посчитаем разницу в днях между двумя повторными покупками и найдем среднее значение этой разницы.

Получим 37 новых столбцов для единой итоговой таблицы.

In [ ]:
# Выпишем все party_rk всех пользователей

df_pr7 = df_socdem['party_rk'].to_frame()

Для каждой из 37 категорий пройдем циклом по всем пользователям и найдем искомое значение признака, затем соберем эти признаки в столбцы и объединим в одну таблицу.

С целью нахождения количества дней между двумя соседними покупками, <strong>построим лаг со сдвигом на один день </strong>. Будем вычитать из начальных дат даты с лагом на один день, получая искомый столбец. Далее будем находить среднее значение по этому столбцу:

In [ ]:
def calc_feature_buyfreq(df_train): 
    ''' 
    FUNC: считает признак - среднее количество дней между транзакциями в категории пользователя 
    IN: df_train :: очищенная на 11 месяцев 
    OUT: cat_ptable = [частота_покупок_Категория1,...,частота_покупок_Категория2, {юзер_id}] 
 
    ''' 
    # считаем проекцию и сортировку  по 'party_rk','category','transaction_dttm' 
    df = df_train[['party_rk','category','transaction_dttm']] 
    df['transaction_dttm'] = pd.to_datetime(df['transaction_dttm']) 
    df = df.sort_values(['category','party_rk','transaction_dttm']) 
    df = df.reset_index().drop(columns=['index']) 
 
    # добавляем строку со сдвигом для 'transaction_dttm','party_rk' 
    df_shifted = df[['transaction_dttm','party_rk']][1:].reset_index().drop(columns=['index']) 
    transaction_dttm_shifted, party_rk_shifted = df_shifted['transaction_dttm'], df_shifted['party_rk'] 
 
    df_shifted = df.iloc[:-1,:] 
 
    df_shifted['transaction_dttm_shifted'] = transaction_dttm_shifted 
    df_shifted['party_rk_shifted'] = party_rk_shifted 
 
    # считаем разницу между изначльным и сдвинутыми columns 
    df_shifted['party_rk_balance'] = df_shifted['party_rk_shifted']  - df_shifted['party_rk'] 
    df_shifted['transaction_dttm_balance'] = (df_shifted['transaction_dttm_shifted']  - df_shifted['transaction_dttm']).dt.days 
    df_shifted.loc[df_shifted['transaction_dttm_balance']<0, 'transaction_dttm_balance'] = 0 
 
    # удаляем строки пользуюясь разницой сдвигов   
    df_shifted = df_shifted[df_shifted['party_rk_balance'] == 0] 
    df_shifted = df_shifted.groupby(['party_rk','category']).mean().drop(columns=['party_rk_shifted','party_rk_balance']).reset_index() 
 
    # считаем признак в cat_ptable 
    cat_ptable = pd.pivot_table(df_shifted, values=['transaction_dttm_balance'], index=['party_rk'], columns=['category']).reset_index().fillna(0.0) 
    cat_ptable.columns = cat_ptable.columns.get_level_values(1) 
 
    # добавляем суфикс '_buyfreq'в columns и column 'party_rk' 
    cat_ptable.columns = pd.Index(list(['categoryii'+str(i)+'_buyfreq' for i in list(cat_ptable.columns)])) 
    cat_ptable['party_rk'] = cat_ptable.iloc[:,:1] 
    cat_ptable = cat_ptable.iloc[:,1:] 
 
    return cat_ptable

def feature5(cat_ptable, matrix3):
    '''
    FUNC: заполняет признак нулями для пользователей, которые ничего не покупали 
    IN: cat_ptable ([частота_покупок_Категория1,...,частота_покупок_Категория2, {юзер_id}])
    IN: matrix3 (таблица с информацией о пользователях)
    OUT: final_feature5 (таблица, строки которой - пользоавтели, столбцы - категории, на пересечении - искомый признак)
    '''
    
    # Выпишем все party_rk всех пользователей
    df_pr5 = matrix3['party_rk'].to_frame()
    df_pr5_ = pd.merge(df_pr5, cat_ptable, left_on='party_rk', right_on='party_rk', how='left')
    final_feature5 = df_pr5_.fillna(0)
    return final_feature5

Получаем таблицу, строки которой - пользоавтели, столбцы - категории, на пересечении - искомый признак)

In [ ]:
f5 = feature5(calc_feature_buyfreq(df_train_out[df_train_out['transaction_dttm'] <= datetime.datetime(2019,11,30)]), df_socdem_num)
f5

### Признак №6 <a class="anchor" id="ch36"></a>

<strong>Признак:   </strong> 
$\frac{\mbox{(Количество покупок у человека в конкретной категории в данном месяце)}}{\mbox{(Общее количество транзакций человека в конкретном месяце)}}$

Посчитаем данный признак поочередно для трёх месяцев. Получим 

$\mbox{Количество категорий}\cdot 3\mbox{ месяца} = 37\cdot3=111$ - новых столбцов для итоговой единой таблицы.

In [ ]:
def cutDate(matrix, start_date, stop_date):
    '''
    FUNC: выводит транзакции в указанном периоде
    IN: matrix (таблица с информацией о транзакциях)
    IN: start_date (дата, с которой начинается необходимый временной промежуток (не включительно))
    IN: stop_date (дата, на которой заканчивается необходимый временной промежуток (включительно))
    OUT: pre_cut (таблица транзакций за указанный период)
    OUT: final_cut (таблица транзакций за указанный период, ключающая только столбец 'party_rk')
    '''
    
    temp1 = matrix[matrix['transaction_dttm'] <= stop_date]
    pre_cut = temp1[start_date <  temp1['transaction_dttm']]
    final_cut = pre_cut['party_rk'].to_frame()
    return pre_cut, final_cut


def findFeature6(matrix, pre_cut, final_cut, df_pr8, num):
    '''
    FUNC: считает значение шестого признака
    IN: matrix (таблица с информацией о транзакциях)
    IN: pre_cut (таблица транзакций за указанный период)
    IN: final_cut (таблица транзакций за указанный период, ключающая только столбец 'party_rk')
    IN: df_pr8 (сборная таблица, в которую последовательно добавляются столбцы с найденными признаками для разных
                месяцев и категорий)
    IN: num (порядковый номер месяца)
    OUT: df_pr8 (сборная таблица, в которую последовательно добавляются столбцы с найденными признаками для разных
                месяцев и категорий)
    '''
    
    temp1 = matrix[matrix['transaction_dttm'] <= datetime.datetime(2019,11,30)]
    temp2 = temp1[datetime.datetime(2019,10,31) < temp1['transaction_dttm']]
    temp3 = temp2['party_rk'].to_frame()
    denominator = temp3['party_rk'].value_counts().to_frame().reset_index()
    
    res8 = pd.DataFrame()
    for i in range(37):
        temp4 = pre_cut[pre_cut['category'] == i]
        temp5 = temp4['party_rk'].to_frame()
        numerator = temp5['party_rk'].value_counts().to_frame().reset_index()
        res6 = denominator.merge(numerator, on='index')
        feature6 = res6['party_rk_y'] / res6['party_rk_x']
        name1 = 'categoryii' + str(i) +'_fracofcat_month' + str(num)
        res6[name1] = feature6
        fin6 = pd.DataFrame()
        fin6['party_rk'] = res6['index']
        fin6[name1] = feature6
        df_pr8 = pd.merge(df_pr8, fin6, left_on='party_rk', right_on='party_rk', how='left')
    return df_pr8


def feature6(matrix, matrix3, date1, date2, date3, date4):
    '''
    FUNC: создает таблицу со значениями искомого признака
    IN: matrix (таблица с информацией о транзакциях)
    IN: matrix3 (таблица с информацией о пользователях)
    IN: date1 (дата окончания третьего из трех месяцев)
    IN: date2 (дата окончания второго из трех месяцев)
    IN: date3 (дата окончания первого из трех месяцев)
    IN: date4 (дата окончания месяца до первого месяца)
    OUT: final_feature8 (таблица, строки которой - пользователи, столбцы - (месяц + категория), на пересечении - значение
                        искомого признака)
    '''
    
    # Выпишем все party_rk всех пользователей
    df_pr8_0 = matrix3['party_rk'].to_frame()
    
    # Запустим функции для получения столбцов-признаков
#     pre_cut1, final_cut1 = cutDate(matrix, date2, date1)
#     pre_cut2, final_cut2 = cutDate(matrix, date3, date2)
#     pre_cut3, final_cut3 = cutDate(matrix, date4, date3)   
    
    temp1 = matrix[matrix['transaction_dttm'] <= date1]
    pre_cut1 = temp1[date2 <  temp1['transaction_dttm']]
    final_cut1 = pre_cut1['party_rk'].to_frame()
    
    temp1 = matrix[matrix['transaction_dttm'] <= date2]
    pre_cut2 = temp1[date3 <  temp1['transaction_dttm']]
    final_cut2 = pre_cut2['party_rk'].to_frame()
    
    temp1 = matrix[matrix['transaction_dttm'] <= date3]
    pre_cut3 = temp1[date4 <  temp1['transaction_dttm']]
    final_cut3 = pre_cut3['party_rk'].to_frame()

    df_pr8_1 = findFeature6(matrix, pre_cut1, final_cut1, df_pr8_0, 1)
    df_pr8_2 = findFeature6(matrix, pre_cut2, final_cut2, df_pr8_1, 2)
    df_pr8_3 = findFeature6(matrix, pre_cut3, final_cut3, df_pr8_2, 3)

    # Заполним нулями все поля, характеризующие пользователя + категорию + месяц, когда покупки вообще не совершались
    final_feature6 = df_pr8_3.fillna(0)
    return final_feature6

Получаем таблицу, столбцами которой являются Один из 3 месяцев + Одна из 37 категорий, строками - пользователи, а на перечении стоит искомый признак:

In [ ]:
f6 = feature6(df_train_out, df_socdem_num, datetime.datetime(2019,11,30), datetime.datetime(2019,10,31), datetime.datetime(2019,9,30), datetime.datetime(2019,8,31))
f6

### Признак №7 (ARIMA) <a class="anchor" id="ch37"></a>

***ARIMA*** (англ. autoregressive integrated moving average, иногда модель Бокса — Дженкинса, методология Бокса — Дженкинса) — интегрированная модель авторегрессии — скользящего среднего — модель и методология анализа временных рядов. Такая модель может интерпретироваться как линейная модель множественной регрессии, в которой в качестве объясняющих переменных выступают прошлые значения самой зависимой переменной, а в качестве регрессионного остатка — скользящие средние из элементов белого шума.

Основная формула модели ARIMA:

𝑋𝑡=𝑐+𝜀𝑡+∑𝑖=1𝑝𝑎𝑖Δ𝑑𝑋𝑡−𝑖+∑𝑗=1𝑞𝑏𝑗𝜀𝑡−𝑗 

𝑋𝑡  - затраты, которые мы хотим предсказать
𝑋𝑡−𝑖  - затраты, основанные на исторических данных, затраты за предыдущие даты
Δ𝑑  - оператор разности временного ряда порядка d (последовательное взятие d раз разностей первого порядка — сначала от временного ряда, затем от полученных разностей первого порядка, затем от второго порядка и т. д.) c — константа
𝜀𝑡  — белый шум
𝑎𝑖,𝑏𝑖  — действительные числа, авторегрессионные коэффициенты и коэффициенты скользящего среднего, соответственно.

В коде у модели 3 параметра: p = порядок авторегресионной модели; d = степень разностей; q = порядок скользящего среднего. 



При помощи ARIMA мы будем **предсказывать затраты пользователей по 37 категорям** на основе временных рядов их транзакций, а далее **уточним эти предсказания через** ... и другие признаки итоговой матрицы X.



In [ ]:
# уберем 12-ый месяц из выборки
df_train_ = df_train_out[pd.DatetimeIndex(df_train_out['transaction_dttm']).month != 12] 

In [ ]:
df_train_['transaction_dttm'] = pd.to_datetime(df_train_['transaction_dttm'])
df_train_ = df_train_.sort_values(by='transaction_dttm')
df_train_.set_index('transaction_dttm', inplace = True)

In [ ]:
%%time

# Предсказание затрат по категории через 4 недели (~ месяц)
def arima(user_cat):
    for t in range(4):
        model = ARIMA(user_cat, order=(5,1,0))
        try:
            model_fit = model.fit()
        except np.linalg.LinAlgError:
            return user_cat.sum() / 13.5 # 54 / 4
        except IndexError:
            return user_cat.sum() / 13.5
        yhat = model_fit.forecast()
        user_cat = user_cat.append(yhat)
    answ = user_cat[-4:].sum()
    if answ < 0:
        condition = user_cat.index >= '2019-10-01'
        answ = user_cat[condition].sum()/3
    return answ


# Объединяем дни в недели
def month_sum(data_series_user):
    answer = data_series_user.resample("1W").sum()
    return answer

# посчитаем ARIMA для временных рядов затрат по категориям у каждому пользователю
def user_cat(data):
    users = data['party_rk'].unique()
    users_list = []
    now_date = str((pd.to_datetime('today') - pd.Timestamp("1970-01-01"))// pd.Timedelta('1s'))
    
    for user in tqdm(users):
            user_list = {'id':user}
            user_data = data[data['party_rk'] == user]
            categories = user_data['category'].unique()
            for category in categories:
                answer_table = user_data[user_data['category'] == category]
                answer_data_series = answer_table['transaction_amt_rur']
                answer_data_series_w = month_sum(answer_data_series)
                if len(answer_data_series_w) == 0:
                    user_list[category] = 0
                else:
                    arima_user_cat = arima(answer_data_series_w)
                    user_list[category] = arima_user_cat
            users_list.append(user_list)
            with open('arima_log'+now_date+'.txt', 'a', encoding='utf-8') as file:
                file.write(str(user_list) + '\n')

        
    return users_list
  

In [ ]:
# здесь мы считаем ариму
# Примерное время, ушедшее на подсчеты, == 37 часов))))))
# %%time
# user_cat_ = user_cat(df_train_)



Поскольку подсчет признаков с помощью ARIMA занимает примерно <strong>37 часов</strong>, мы сделали это заранее и загрузили результаты в файл <strong>df8</strong>.

Откроем его и будем использовать далее:

In [ ]:
df8=pd.read_csv(r'C:\ВШЭ 3 курс\FINOdays\tinkoff_hackathon_data\tinkoff_hackathon_data\df_arima_all.csv')

In [ ]:
df8 = df8.set_index('party_rk')

In [ ]:
df8

## Создание единой таблицы  <a class="anchor" id="ch4"></a>

Создадим единую таблицу, объединив все признаки, которые были найдены <strong>(Признак №1 - Признак №7)</strong>, а также таблицы <strong>df_product</strong> и <strong>df_socdem</strong>.

In [ ]:
df1 = pd.merge(df_socdem_num_, df_products)
df1

In [ ]:
df2 = pd.merge(df1, f1)
df2

In [ ]:
df3 = pd.merge(df2, f2)
df3

In [ ]:
df4 = pd.merge(df3, f3)
df4

In [ ]:
df5 = pd.merge(df4, f4)
df5

In [ ]:
df6 = pd.merge(df5, f5)
df6

In [ ]:
df7 = pd.merge(df6, f6)
df7

In [ ]:
df7 = df7.set_index('party_rk')

In [ ]:
df7

In [ ]:
df9 = df7.merge(df8,on = 'party_rk')

Приведем таргет к необходимому формату

In [ ]:
target_full = df_train_out[pd.DatetimeIndex(df_train_out['transaction_dttm']).month == 12]

In [ ]:
ad_users = pd.DataFrame(data = {'party_rk':pd.Series(list(set(df8.index.tolist())-set(target_full['party_rk'].unique())))},columns = target_full.columns).fillna(0)

In [ ]:
ad_users['category'] = np.array([0]*ad_users['category'].shape[0])

In [ ]:

target_full = pd.concat([target_full,ad_users])

In [ ]:

target_full = target_full[target_full.party_rk.isin(df8.index.tolist())]

In [ ]:
target_full['party_rk'].unique().shape

In [ ]:
final_target_full = target_full.pivot_table(index = ['party_rk'],columns = ['category'],values = ['transaction_amt_rur'],aggfunc = 'sum').fillna(0)

In [ ]:
final_target_full.columns = final_target_full.reset_index().set_index('party_rk').columns.get_level_values(1)

Присоединим target_full к таблице признаков

In [ ]:
df9 = df9.merge(final_target_full,on='party_rk')

In [ ]:
df9

In [ ]:
# pd.read_csv(r'C:\ВШЭ 3 курс\FINOdays\tinkoff_hackathon_data\tinkoff_hackathon_data\arima_target.csv')

In [ ]:
####### загрузила этот файл в разделе с аримой
# df8=pd.read_csv(r'C:\ВШЭ 3 курс\FINOdays\tinkoff_hackathon_data\tinkoff_hackathon_data\full_frame.csv')

In [ ]:
# df8 = df8.set_index('party_rk')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# del final_target_full

In [ ]:
# del target_full

In [ ]:
# df9 = df7.merge(df8,on = 'party_rk')

In [ ]:
# df9 = df9.merge(final_target_full,on='party_rk')

In [ ]:
# import random

In [ ]:
# random.seed = 42

## Предсказательная модель  <a class="anchor" id="ch5"></a>

In [ ]:
# x_train,x_test,y_train,y_test = train_test_split(df9.iloc[:,:-36],df9.iloc[:,-36:],test_size = 0.2,random_state = 42)

In [ ]:

import numpy as np

from sklearn.multioutput import MultiOutputRegressor

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA

In [ ]:
x_train = df9.iloc[:,:-36].fillna(0)
y_train = df9.iloc[:,-36:]

In [ ]:
x_test = x_test.fillna(0)

In [ ]:
x_train = x_train.fillna(0)

In [ ]:
RF = RandomForestRegressor(max_depth=5,n_estimators = 500,n_jobs=-1,random_state=42)

In [ ]:
%%time
RF.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import r2_score

## Метрики задач машинного обучения, соответствующие цели задачи <a class="anchor" id="ch6"></a>

In [ ]:
df_test = pd.read_csv(r'C:\ВШЭ 3 курс\FINOdays\tinkoff_hackathon_data\tinkoff_hackathon_data\avk_hackathon_data_test_transactions.csv')


Возможные метрики машинного обучения, применимые к нашей задачи: \
MSE  \
MAE \
RMSE \
Huber-Loss \
R-квадрат


Почему мы выбрали R-квадрат?
Его значения лежат в интервале от 0 до 1, поэтому его результаты интерпретируемы 
У используемых нами моделей есть метод score(X, y[, sample_weight]), который возвращает коэффициент детерминации прогноза


## Тестовая выборка <a class="anchor" id="ch7"></a>

### Общая идея <a class="anchor" id="ch71"></a>

Используя данные о транзакциях за 1 год (тренировочная выборка), предскажем затраты пользователей по категориям на следующий месяц (на первый месяц тестовой выборки):

Используем данные за: | Получаем прогноз на:
--- | --- 
01.01.2019 - 31.12.2019 | 01.01.2020 - 31.01.2020

Затем, используя данные о транзакциях за 1 год и 1 месяц (тренировочная выборка + 1 месяц тестовой), предскажем затраты пользователей по категориям на следующий месяц (на второй месяц тестовой выборки):

Используем данные за: | Получаем прогноз на:
--- | --- 
01.01.2019 - 31.01.2020 | 01.02.2020 - 29.02.2020

В таблице df_train содержится информация о совершенных пользователями транзакциях, где
<ul>
<li><strong>party_rk</strong> - уникальный идентификационный номер клиента</li> 
<li><strong>account_rk</strong> - уникальный идентификационный номер счета</li>
<li><strong>financial_account_type_cd</strong> - тип карты (дебетовая/кредитовая)</li>
<li><strong>transaction_dttm</strong> - день совершения операции в формате ГГГГ-ММ-ДД</li>
<li><strong>transaction_amt_rur</strong> - сумма транзакции в рублях</li>
<li><strong>merchant_rk	</strong> - идентификационный номер мерчант-аккаунта (оборудования для приема платежей)</li>  
<li><strong>merchant_type</strong> - идентификационный номер типа мерчанта</li> 
<li><strong>merchant_group_rk</strong> - идентификационный номер мерчант-группы, обслуживающей соответствующий мерчант-аккаунт</li>
<li><strong>category	</strong> - категория транзакции</li>  
</ul>

In [ ]:
df_test['category'] = df_test['category'].fillna(df_test['category'].mode()[0])

In [ ]:
df_test['party_rk'].unique().shape

In [ ]:
final_target_full_test = df_test.pivot_table(index = ['party_rk'],columns = ['category'],values = ['transaction_amt_rur']).fillna(0)
final_target_full_test.columns = final_target_full_test.reset_index().set_index('party_rk').columns.get_level_values(1)

In [ ]:
print('Количество строк: {}, количество столбцов: {}'.format(df_test.shape[0], df_test.shape[1]))

In [ ]:
final_target_full_test

In [ ]:
df_test.info()

### Обработка данных <a class="anchor" id="ch72"></a>

Переведем столбец с датами транзакций в тип datetime:

In [ ]:
df_test['transaction_dttm'] = pd.to_datetime(df_test['transaction_dttm'])

In [ ]:
df_test.describe()

Повторим аналогичные действия, как и с таблицей df_train: приведем категориальные признаки к числовому виду и уберем не нужные для прогноза столбцы.

In [ ]:
df_test_clean = df_test.drop(['account_rk','merchant_rk','merchant_group_rk', 'merchant_type', 'transaction_type_desc'], axis = 1)


In [ ]:
del df_test_clean['category']

df_test_clean.head()


Поделим df_train на две части: 
<ul>
    <li><strong>df_test_clean_1</strong> -  информация обо всех транзакциях за первый месяц тестовой выборки -----> с 01.01.2020 по 31.01.2020</li>
    <li><strong>df_test_clean_2</strong> -  информация обо всех транзакциях за второй месяц тестовой выборки -----> с 01.02.2020 по 29.02.2020</li>
    </ul>

In [ ]:
df_test_clean['Month'] = pd.DatetimeIndex(df_test_clean['transaction_dttm']).month
df_test_clean_1 = df_test_clean[df_test_clean['Month'] == 1]
df_test_clean_2 = df_test_clean[df_test_clean['Month'] == 2]

In [ ]:
del df_test_clean_1['Month']
del df_test_clean_2['Month']

In [ ]:
# del df_test_clean_1['transaction_amt_rur']

In [ ]:
df_test_clean_1

### Подсчет признаков для предсказания на первый месяц <a class="anchor" id="ch73"></a>

Создадим таблицы с информацией о транзакциях:
<ul>
    <li><strong>df_train_test1</strong> - транзакции за год (из тренировочной выборки) + 1 месяц (из тестовой выборки) -----> с 01.01.2019 по 31.01.2020</li>
    <li><strong>df_train_test</strong> - транзакции за год (из тренировочной выборки) + 2 месяца (из тестовой выборки) -----> с 01.01.2019 по 29.02.2020</li>
</ul>

In [ ]:
df_train_test1 = pd.concat([df_train_out, df_test_clean_1], axis=0 )
df_train_test = pd.concat([df_train_out, df_test], axis=0 )

Посчитаем дополнительные признаки (Признак №1 - Признак №8) для таблицы <strong>df_train_test_1</strong>. 

Затем соединим в одну таблицу <strong>df_train_test1_final</strong> все получившиеся признаки, а также таблицы df_socdem_num и df_product.

In [ ]:
# соединим таблицы df_socdem_num и df_product
df_test_1_temp_ = pd.merge(df_socdem_num, df_products)

# feature 1
df_test_1_temp1 = pd.merge(df_test_1_temp_, f1)

# feature 2
df_test_1_temp2 = pd.merge(df_test_1_temp1, f2)

# feature 3
df_test_1_temp3 = pd.merge(df_test_1_temp2, feature3(df_train_out, df_socdem_num, datetime.datetime(2019,12,31), datetime.datetime(2019,9,30)))

# feature 4
trans_by11month_1 = calc_trans_by11month(df_train_test1)
df_test_1_temp4 = pd.merge(createFeature4(trans_by11month_1, df_socdem_num), df_test_1_temp3)

# feature 5
df_test_1_temp5 = pd.merge(feature5(calc_feature_buyfreq(df_train_out), df_socdem_num), df_test_1_temp4)

# feature 6
df_test_1_temp6 = pd.merge(feature6(df_train_out, df_socdem_num, datetime.datetime(2019,12,31), datetime.datetime(2019,11,30), datetime.datetime(2019,10,31), datetime.datetime(2019,9,30)), df_test_1_temp5)

# # feature 7
df_train_test1_final = pd.merge(df_test_1_temp6, df8.reset_index()).set_index('party_rk')

In [ ]:
total_dataset_test_1 = df_train_test1_final.merge(final_target_full_test,on = 'party_rk')

In [ ]:
total_dataset_test_1 = pd.get_dummies(total_dataset_test_1)

In [ ]:
total_dataset_test_1 = total_dataset_test_1.iloc[:,:-36].fillna(0)

In [ ]:
pred_test_1 = RF.predict(total_dataset_test_1.iloc[:,:-1])

In [ ]:
r2_score(total_dataset_test_1.iloc[:,-36:],pred_test_1,multioutput='variance_weighted')

### Подсчет признаков для предсказания на второй месяц <a class="anchor" id="ch74"></a>

Посчитаем дополнительные признаки (Признак №1 - Признак №8) для таблицы <strong>df_train_test</strong>. 

Затем соединим в одну таблицу <strong>df_train_test_final</strong> все получившиеся признаки, а также таблицы df_socdem_num и df_product.

In [ ]:
# соединим таблицы df_socdem_num и df_product
df_test_2_temp = pd.merge(df_socdem, df_products)

# feature 1
df_test_2_temp1 = pd.merge(df_test_2_temp, f1)

# feature 2
df_test_2_temp2 = pd.merge(df_test_2_temp1, f2)

# feature 3
df_test_2_temp3 = pd.merge(df_test_2_temp2, feature3(df_train_test1, df_socdem_num, datetime.datetime(2020,1,31), datetime.datetime(2019,10,31)))

# feature 4
trans_by11month_2 = calc_trans_by11month(df_train_test)
df_test_2_temp4 = pd.merge(createFeature4(trans_by11month_2, df_socdem_num), df_test_2_temp3)

# feature 5
df_test_2_temp5 = pd.merge(feature5(calc_feature_buyfreq(df_train_test1), df_socdem_num), df_test_2_temp4)

# feature 6
df_test_2_temp6 = pd.merge(feature6(df_train_test1, df_socdem_num, datetime.datetime(2020,1,31), datetime.datetime(2019,12,31), datetime.datetime(2019,11,30), datetime.datetime(2019,10,31)), df_test_2_temp5)

# feature 7
df_train_test2_final = pd.merge(df_test_2_temp6, df8.reset_index()).set_index('party_rk')

In [ ]:

total_dataset_test_2 = df_train_test2_final.merge(final_target_full_test,on = 'party_rk')

In [ ]:
total_dataset_test_2 = total_dataset_test_2.drop('gender_cd',axis = 1)

In [ ]:
total_dataset_test_2 = pd.get_dummies(total_dataset_test_2)

In [ ]:
total_dataset_test_2 = total_dataset_test_2.iloc[:,:-36].fillna(0)

In [ ]:
pred_test_2 = RF.predict(total_dataset_test_2.iloc[:,:-107])

In [ ]:
r2_score(total_dataset_test_2.iloc[:,-36:],pred_test_2,multioutput='variance_weighted')

## Тестирование модели на тестовых данных <a class="anchor" id="ch8"></a>

Протестируем модель на тестовых данных. Сначала найдем <strong>прогноз на первый месяц</strong> тестовой выборки (01.01.2020 - 31.01.2020). Для этого воспользуемся построенным ранее датафреймом df_train_test1_final, в котором собраны все необходимые признаки.

In [ ]:
df_train_test1_final

Протестируем модель на тестовых данных. Сначала найдем <strong>прогноз на второй месяц</strong> тестовой выборки (01.02.2020 - 29.02.2020). Для этого воспользуемся построенным ранее датафреймом df_train_test2_final, в котором собраны все необходимые признаки.

In [ ]:
df_train_test2_final

## Выбор бизнес-метрик  <a class="anchor" id="ch9"></a>

<strong>Разработаем</strong> бизнес-метрики для нашей системы предсказания трат пользователя по категориям, а затем <strong>посчитаем значения</strong> этих метрик по предложенным данным.  

<strong>Система кэшбека выгодна банкам </strong>по нескольким причинам, основная из которых:
\
кэшбек стимулирует клиентов чаще пользоваться картой и хранить на ней как можно больше денег, которые в это время может использовать банк в своих целях.

Если наша система предсказаний затрат клиентов по категориям сработает эффективно, <strong>средний кэшбек покупок возрастет</strong>. Клиент, которому порекомендовали действительно актуальные для него категории кэшбека, начнет <strong>тратить больше денег</strong> на покупки по данным категориям. 
 

В связи с этим необходимо разработать бизнес-метрики, позволяющие измерить, <strong>насколько изменились траты и удовлетворенность</strong> пользователей после введения системы предсказаний.

<strong>Основные требования</strong> к метрикам для системы предсказания затрат пользователя по категориям:
<ol> 
<li> При эффективной системе предсказаний клиент не снизит расходы по остальным категориям, по которым он не получает кэшбек. По этой причине содержательные бизнес-метрики должны учитывать показатели по всем категориям, а не только по тем трем, на которые распространяется кэшбек. 
</li>
<li> Метрики должны отражать относительные величины, а не абсолютные. Это важно, поскольку мы хотим сравнивать показатели метрик за различные периоды времени. </li></ol> 


### Бизнес-метрика №1 <a class="anchor" id="ch91"></a>

<u><strong>Прирост средней суммы затрат по всем категориям у пользователя за месяц (среднее значение по всем пользователям)</strong></u>

Введем следующие обозначения: пусть $j$- номер пользователя, $j=1,\ldots,50000.\;\;$ Пусть

Номер месяца | Средняя сумма затрат по всем категориям у j-го пользователя 
--- | --- 
$t-1$ | $X_{j_{t-1}}$ 
$t$ | $X_{j_{t}}$  

Тогда бизнес-метрику по одному ($M_{1j}$) и по всем ($M_1$) пользователям посчитаем по следующим формулам:

Бизнес-метрика по одному пользователю | Усредненная бизнес-метрика по всем пользователям
 --- | ---
$M_{1_j}=\frac{X_{j_{t}}-X_{j_{t-1}}}{X_{j_{t-1}}}$ | $M_1 = \frac{\sum_{j=1}^{n}M_{1_j}}{n}$


где $n$ - количество пользователей (50000). 

В результате применения системы предсказания, затраты пользователей должны возрасти, следовательно, <strong>увеличится и значение $M_1$</strong>. Такая <strong>бизнес-метрика удобна для</strong> измерения эффективности внедрения каких-либо новых систем и применения новых стратегий.

#### Подсчёт бизнес-метрики <a class="anchor" id="ch911"></a>

Напишем необходимые функции:

In [ ]:
def cutDateM1_(start_date, stop_date, matrix):
    '''
    FUNC: выводит транзакции в указанном периоде
    IN: start_date (дата, с которой начинается необходимый временной промежуток (включительно))
    IN: stop_date (дата, на которой заканчивается необходимый временной промежуток (включительно))
    IN: matrix (таблица с информацией о транзакциях)
    OUT: pre_cut (таблица транзакций за указанный период)
    OUT: final_cut (таблица транзакций за указанный период, ключающая только столбец 'party_rk')
    '''
    
    v1_1 = matrix[matrix['transaction_dttm'] <= stop_date]
    pre_cut = v1_1[start_date <= v1_1['transaction_dttm']]
    final_cut = pre_cut['party_rk'].to_frame()
    return pre_cut, final_cut


def findMeanDf(df_t):
    '''
    FUNC: находит среднее значение затрат по пользователю
    IN: df_t (таблица транзакций за указанный период)
    OUT: df_mean_amt_ (таблица со столбцом средних затрат за период по каждому пользователю)
    '''
    df_train_rk_amt = df_t[['party_rk', 'transaction_amt_rur']]
    df_mean_amt = df_train_rk_amt.groupby(['party_rk']).mean()
    df_mean = df_mean_amt.reset_index()
    df_mean_amt_ = pd.merge(df_pr8_0, df_mean, how='left')
    df_mean_amt_ = df_mean_amt_.fillna(0)
    return df_mean_amt_

# Считаем M1j 
def countM1j(df_m):
    '''
    FUNC: находит значение M1j для пользователей
    IN: df_m (таблица со столбцом средних затрат за период по каждому пользователю)
    OUT: df_m (таблица со столбцом средних затрат за период по каждому пользователю + посчитанная для него M1j)
    '''
    df_m['m1j'] = (df_m['transaction_amt_rur_y'] - df_m['transaction_amt_rur_x']) / df_m['transaction_amt_rur_x']
    return df_m

# Считаем M1
def countM1(df_m1j):
    '''
    FUNC: находит значение M1 
    IN: df_m1j (таблица со столбцом средних затрат за период по каждому пользователю + посчитанная для него M1j)
    OUT: M1 (значение метрики)
    '''
    M1 = df_m1j['m1j'].mean()
    return M1

Посчитаем значение $M_{1_{before}}$ бизнес-метрики за первые 2 месяца тренировочной выборки, а именно оценим прирост средней суммы затрат за период с 01.01.2019 по 28.02.2019:

In [ ]:
# Выпишем все party_rk всех пользователей
df_pr8_0 = df_socdem_num['party_rk'].to_frame()

# Найдем X_{j_{t-1}} (за январь)
df_01_02_2019, final_cut0102 = cutDateM1_(datetime.datetime(2019, 1, 1), datetime.datetime(2019, 1, 31), df_train_out)
df_01_02_2019_with_mean = findMeanDf(df_01_02_2019)

# Найдем X_{j_{t}} (за февраль)
df_02_03_2019, final_cut0203 = cutDateM1_(datetime.datetime(2019, 2, 1), datetime.datetime(2019, 2, 28), df_train_out)
df_02_03_2019_with_mean = findMeanDf(df_02_03_2019)

# Объединим за январь и февраль
df_m1_b = pd.merge(df_01_02_2019_with_mean, df_02_03_2019_with_mean, on='party_rk')
df_m1_b

In [ ]:
# Удалим строчки, в которых и средние траты за январь, и средние траты за февраль равны 0
# (такие пользователи просто не пользовались картой в этих месяцах)

df_m1_b = df_m1_b[~((df_m1_b.transaction_amt_rur_x == 0) & (df_m1_b.transaction_amt_rur_x == 0))]
df_m1_b

In [ ]:
# Посчитаем значение M1 для каждого из пользователей (получим M1j)
df_m1_b_ = countM1j(df_m1_b)
df_m1_b_

In [ ]:
# Посчитаем усредненное значение метрики M1
M1_before = countM1(df_m1_b_)
M1_before

Посчитаем значение $M_{1_{after}}$ бизнес-метрики за последний месяц тренировочной выборки и первый месяц тестовой выборки, а именно оценим прирост средней суммы затрат за период с 01.12.2019 по 31.01.2020:

In [ ]:
# Выпишем все party_rk всех пользователей
df_pr8_0 = df_socdem_num['party_rk'].to_frame()

# Найдем X_{j_{t-1}} (за декабрь)
df_12_01_2019, final_cut1201 = cutDateM1_(datetime.datetime(2019, 12, 1), datetime.datetime(2019, 12, 31), df_train_test)
df_12_01_2019_with_mean = findMeanDf(df_12_01_2019)

# Найдем X_{j_{t}} (за январь)
df_01_02_2020, final_cut0102 = cutDateM1_(datetime.datetime(2020, 1, 1), datetime.datetime(2020, 1, 31), df_train_test)
df_01_02_2020_with_mean = findMeanDf(df_01_02_2020)

# Объединим за декабрь и январь
df_m1_a = pd.merge(df_12_01_2019_with_mean, df_01_02_2020_with_mean, on='party_rk')
df_m1_a

In [ ]:
# Удалим строчки, в которых и средние траты за декабрь, и средние траты за январь равны 0
# (такие пользователи просто не пользовались картой в этих месяцах)

df_m1_a = df_m1_a[~((df_m1_a.transaction_amt_rur_x == 0) & (df_m1_a.transaction_amt_rur_x == 0))]
df_m1_a

In [ ]:
# Посчитаем знаечние M1 для каждого из пользователей (получим M1j)
df_m1_a = countM1j(df_m1_a)
df_m1_a

In [ ]:
# Посчитаем усредненное значение метрики M1
M1_after = countM1(df_m1_a)
M1_after

### Бизнес-метрика №2 <a class="anchor" id="ch92"></a>

<u><strong>ARPU (Average Revenue Per User)</strong></u> - средняя выручка на одного пользователя. 

В общем виде метрика записывается следующим образом:  $ARPU=\frac{\mbox{общая выручка со всех продаж}}{\mbox{количество покупателей}}$.

Метрику $ARPU$ зачастую применяют для того, чтобы проверить эффективность нового канала привлечения, в том числе, эффективность введения системы предсказания затрат.

<strong>В нашем кейсе</strong> значение данной метрики нужно считать отдельно <strong>для каждой из 37-ми категорий</strong>, так как общую выручку со всех продаж и количество покупателей различных категорий нельзя сравнивать между собой с точки зрения смысла.  

Поскольку среднее время жизни пользователя банка - велико, мы можем считать ARPU за период больше месяца. Рассмотрим данную метрику применительно к периоду из <strong>2 месяцев</strong>:

\begin{eqnarray}
M_2 = ARPU_i\mbox{ (за 2 мес.)}=\frac{\mbox{общая выручка со всех продаж в i-ой категории}}{\mbox{количество покупателей в i-ой категории}}
\end{eqnarray}

Данная метрика $M_2$ позволит <strong>сравнить среднюю выручку</strong> с одного пользователя <strong>за различные периоды времени</strong>: до ввода системы предсказаний и после. Повышение $M_2$ будет говорить о какой-либо из следующих закономерностей в рассматриваемой категории:
<ul>
    <li>о <strong>росте количества клиентов, </strong></li>
    <li>об <strong>увеличении среднего чека</strong>.</li>
    </ul>

#### Подсчёт бизнес-метрики <a class="anchor" id="ch921"></a>

Напишем необходимые функции:

In [ ]:
def arpu(matrix1, begin_date, end_date, a):
    '''
    FUNC: считает метрику ARPU
    IN: matrix1 (таблица, откуда мы берем данные)
    IN:begin_date (дата начала рассматриваемого периода) 
    IN:end_date (дата конца рассматриваемого периода)
    IN: a (string переменная, название столбцов)
    OUT: df_final (таблица с категориями и соответствующими ARPU метриками)
    '''
    # создадим таблицу, соответствующую нашему отрезку времени
    df_arpu = matrix1[(matrix1['transaction_dttm'] <= end_date) 
                                 & (matrix1['transaction_dttm'] >= begin_date)]
    
    # делаем цикл по категориям и считает для каждой категории ARPU метрики
    arpus = []
    categories = matrix1['category'].unique()
    for i in categories:
        df_arpu_cat = df_arpu[df_arpu['category'] == i]
        revenue_all = np.sum(df_arpu_cat['transaction_amt_rur'])
        num_users = len(df_arpu_cat['party_rk'].unique())
        arpus.append(revenue_all/num_users)
    column_name = 'ARPU_' + a
    df_final = pd.DataFrame({'category': categories, column_name : arpus})
    
    return df_final

Посчитаем значение $M_{2_{before}}$ бизнес-метрики за первые 2 месяца тренировочной выборки, а именно за период с 01.01.2019 по 28.02.2019:

In [ ]:
arpus_first = arpu(df_train_out, '01.01.2019', '28.02.2019', 'first_2months')

Посчитаем значение $M_{3_{after}}$ бизнес-метрики за последние 2 месяца (из тестовой выборки), а именно за период с 01.01.2020 по 29.02.2020:

In [ ]:
arpus_last = arpu(df_test, '01.01.2020', '29.02.2020', 'last_2months')

In [ ]:
arpu_final = pd.merge(arpus_first, arpus_last )
arpu_final

### Бизнес-метрика №3 <a class="anchor" id="ch93"></a>

<u><strong>LTV (Life Time Value)</strong></u> - пожизненная стоимость клиента.  

В общем виде метрика записывается следующим образом:  $LTV=\mbox{средний чек}\cdot\mbox{количество месяцев, в которых клиент пользуется сервисом}$.

<strong>В нашем кейсе</strong> значение данной метрики нужно считать отдельно <strong>для каждой из 37-ми категорий</strong>, так как чек и количество месцев по различным категориям нельзя сравнивать между собой с точки зрения смысла.  

Пусть $LTV_{i}(\mbox{ по месяцу})$ - это значение $LTV$ по i-ой категории, за выбранные для измерения 2 месяца, где $i=1,\ldots,37$.  

Пусть $LTV_{ij}(\mbox{ по месяцу})$ - это значение $LTV$ по i-ой категории у j-го человека, за выбранные для измерения 2 месяца.  

<strong>Неусредненное значение</strong> этой метрики в подсчете на j-го пользователя:  

$LTV_{ij}(\mbox{ по месяцу})=\mbox{(средний чек по i-ой категории)}\cdot\mbox{ (количество месяцев, где клиент пользуется сервисом на данный момент)}$.

Тогда <strong>усредненное значение</strong> этой метрики для i-ой категории (в подсчете на всех пользователей):  

\begin{eqnarray}
M_3=LTV_{i}\mbox{ (за месяц})=\frac{\sum_{j=1}^{n}LTV_{ij}}{n},\;\;
\end{eqnarray}
 где $n=50000$ - число пользователей.

$M_3$ отображает сумму за месяц, которую один усредненный пользователь тратит на рассматриваемую категорию. 

Увеличение $M_3$ сможет говорить о каких-либо из следующих закономерностей в рассматриваемой категории:
<ul>
    <li>о <strong>увеличении объема покупок </strong>пользователей,</li>
    <li>об <strong>удержании пользователей</strong>, которые и без кэшбека покупают товары категории,</li>
    <li>о <strong>росте количества пользователей</strong>, покупающих товары категории.</li></ul>


### Дополнительная бизнес-метрика <a class="anchor" id="ch94"></a>

Заметим, что в таблице из файла 'avk_hackathon_data_story_texts.csv' <strong>есть много историй</strong>, заголовок (name) и содержание (story_text) которых содержит <strong>слово "Кэшбек"</strong> в различном контексте. 

В связи с этим наблюдением можно создать <strong>дополнительную (второстепенную) бизнес-метрику</strong>, характеризуюющую удовлетворенность клиентов сервисом для подбора категорий кэшбека. 

Если система предсказания категорий сработала хорошо и предложила пользователю кэшбек по его любимым категориям, то человек <strong>заинтересуется системой кэшбека</strong>, даже если раньше он никогда ею не пользовался. 

В таком случае, возможно, он начнет ставить реакции <strong>"like" или "favorite" на истории со словом "Кэшбек"</strong>.

Введем следующие обозначения: пусть $j$- номер пользователя, $j=1,\ldots,50000.\;\;$ Пусть

Номер месяца | Количество реакций "like" или "favorite" на истории со словом "Кэшбек" в name или story_text
--- | --- 
$t-1$ | $Y_{j_{t-1}}$ 
$t$ | $Y_{j_{t}}$  

Тогда бизнес-метрику по одному ($M_{4j}$) и по всем ($M_4$) пользователям посчитаем по следующим формулам:

Бизнес-метрика по одному пользователю | Усредненная бизнес-метрика по всем пользователям
 --- | ---
$M_{4_j}=\frac{Y_{j_{t}}-Y_{j_{t-1}}}{Y_{j_{t-1}}}$ | $M_4 = \frac{\sum_{j=1}^{n}M_{4_j}}{n}$


где $n$ - количество пользователей (50000). (Если $Y_{j_{t-1}}=0$, то пусть $M_{4_j} = Y_{j_{t}}$.)

Данную метрику стоит посчитать для любой <strong>пары месяцев до введения</strong> системы предсказания расходов ($M_{4_{before}}$) и для <strong>пары месяцев сразу же после ее введения</strong> ($M_{4_{after}}$). Если система сработала эффективно и люди удовлетворены сервисом, то 
\begin{eqnarray}
M_{4_{before}} < M_{4_{after}}.
\end{eqnarray}

Данную метрику стоит посчитать для любой <strong>пары месяцев до введения</strong> системы предсказания расходов ($M_{4_{before}}$) и для <strong>пары месяцев сразу же после ее введения</strong> ($M_{4_{after}}$). Если система сработала эффективно и люди удовлетворены сервисом, то 
\begin{eqnarray}
M_{4_{before}} < M_{4_{after}}.
\end{eqnarray}

## Возможные улучшения <a class="anchor" id="ch10"></a>

Мы видим несколько различных возможных улучшений для разработанного нами сервиса:

<ol>
    <li><strong>Кластеризация пользователей </strong></li> - 
Имея данные о каждом пользователе, мы можем сформировать кластеры, которые будут характеризовать определённые особенности поведения пользователей. Клиенты в этих кластерах ведут себя примерно одинаково. Для каждого кластера посчитаем средние затраты пользователей по каждой категории. Модель, принимая на вход нового пользователя, определяет его в ближайший кластер и присваивает ему те затраты по каждой категории, которые являются средними по пользователям этого кластера.
    <li><strong>LDA</strong></li> - LDA в основном применяется в задачах «Topic modeling». С помощью него мы можем обработать таблицу с текстами, которая у нас названа df_stories_texts и выделить ключевые слова из каждой стори. Затем мы можем понять, какими stories интересуются пользователей. На основе его предпочтений и истории лайков/дизлайков/любимых мы можем предсказать ему ту или иную категорию.
    <li><strong>pmdarima</strong></li> - эта библиотека является улучшенной версией той, которую мы используем для обработки временных рядов в этом проекте. С помощью нее возможно получение более точной модели.
    </ol>

<strong>Спасибо за внимание!</strong>